# Import Data Douane To ElasticSearch

In [1]:
import pandas as pd
import os
import elasticsearch
from elasticsearch import Elasticsearch
import elasticsearch_dsl

In [2]:
my_dir = r'C:/Users/aaras/Desktop/Data_Douane'
os.chdir(my_dir)

In [3]:
df_export = pd.DataFrame(columns=['Flux', 'Mois', 'Annee', 'Code_CPF6','Code_A129','Code_NC8','Code_Pays','Valeur', 'Masse', 'USUP'])

In [4]:
import_export_dir = os.path.join(my_dir,'import_export')
os.chdir(import_export_dir)
for file in os.listdir(import_export_dir):
    df_export = pd.concat([df_export, pd.read_csv(file,sep=";",header=None,names=['Flux', 'Mois', 'Annee', 'Code_CPF6','Code_A129','Code_NC8','Code_Pays','Valeur', 'Masse', 'USUP'],dtype={'Flux': object , 'Mois': int , 'Annee': int, 'Code_CPF6': object, 'Code_A129': object, 'Code_NC8':object,'Code_Pays':object, 'Valeur':int, 'Masse':int, 'USUP':int})])
    print('Document {} ajoutée au dataframe'.format(file))

Document National_2010_Export.txt ajoutée au dataframe
Document National_2010_Import.txt ajoutée au dataframe
Document National_2011_Export.txt ajoutée au dataframe
Document National_2011_Import.txt ajoutée au dataframe
Document National_2012_Export.txt ajoutée au dataframe
Document National_2012_Import.txt ajoutée au dataframe
Document National_2013_Export.txt ajoutée au dataframe
Document National_2013_Import.txt ajoutée au dataframe
Document National_2014_Export.txt ajoutée au dataframe
Document National_2014_Import.txt ajoutée au dataframe
Document National_2015_Export.txt ajoutée au dataframe
Document National_2015_Import.txt ajoutée au dataframe
Document National_2016_Export.txt ajoutée au dataframe
Document National_2016_Import.txt ajoutée au dataframe
Document National_2017_Export.txt ajoutée au dataframe
Document National_2017_Import.txt ajoutée au dataframe
Document National_2018_Export.txt ajoutée au dataframe
Document National_2018_Import.txt ajoutée au dataframe
Document N

In [6]:
df_export.shape[0]

40143407

In [18]:
df_export.loc[df_export['Code_Pays'].isnull()]

,Flux,Mois,Annee,Code_CPF6,Code_A129,Code_NC8,Code_Pays,Valeur,Masse,USUP
17201,E,1,2010,105140,C10E,04069099,NaN,1911,99,0
28373,E,1,2010,110610,C11Z,11071099,NaN,109620,378000,0
34913,E,1,2010,139214,C13Z,63026000,NaN,2125,60,0
47241,E,1,2010,141922,C14Z,62114390,NaN,24000,404,0
74258,E,1,2010,205310,C20C,33021090,NaN,5610,490,0
...,...,...,...,...,...,...,...,...,...,...
1680503,I,12,2019,323015,C32C,95069990,NaN,312,4,0
1680962,I,12,2019,324031,C32C,95030010,NaN,143,14,0
1681349,I,12,2019,324039,C32C,95030099,NaN,821,138,0
1685543,I,12,2019,329959,C32C,99050000,NaN,705,0,0


In [19]:
df_null = df_export.loc[df_export['Code_Pays'].isnull()]

In [ ]:
additional_data_dir = os.path.join(my_dir,'additional_data')
os.chdir(additional_data_dir)
df_a129 = pd.read_csv('Libelle_A129.txt', sep=";", header=None, names = ['Code_A129','Libelle_A129','c','d'], usecols = ['Code_A129','Libelle_A129'], dtype={'Code_A129':object,'Libelle_A129':object})
df_cpf6 = pd.read_csv('Libelle_CPF6.txt', sep=";", header=None, names = ['Code_CPF6', 'Libelle_CPF6','c','d'], usecols = ['Code_CPF6', 'Libelle_CPF6'],dtype={'Code_CPF6':object,'Libelle_CPF6':object})
df_nc8 = pd.read_csv('Libelle_NC8_2019.txt', sep=";", header=0,encoding = 'latin-1', names = ['Code_NC8','Libelle_NC8',"Présence d'unité supplémentaire" , "Nature de l'unité supplémentaire","Debut","Fin"], usecols = ['Code_NC8','Libelle_NC8'], dtype={'Code_NC8':object,'Libelle_NC8':object} )
df_pays = pd.read_csv('Libelle_PAYS.txt', sep=";", header=None, names = ['Code_Pays', 'Libelle_Pays', 'c','d'],usecols=['Code_Pays', 'Libelle_Pays'], dtype={'Code_Pays':object,'Libelle_Pays':object})


In [ ]:
df_export_new = pd.merge(df_export, df_a129, how='left')
print('Libellés A129 ajoutés')
print(df_export_new.shape[0])
df_export_new = pd.merge(df_export_new, df_cpf6,how='left')
print('Libellés CPF6 ajoutés')
print(df_export_new.shape[0])
df_export_new = pd.merge(df_export_new, df_nc8,how='left')
print('Libellés NC8 ajoutés')
print(df_export_new.shape[0])
df_export_new = pd.merge(df_export_new, df_pays,how='left')
print('Libellés Pays ajoutés')
print(df_export_new.shape[0])
new_columns_order = ['Flux', 'Mois', 'Annee', 'Code_CPF6', 'Libelle_CPF6', 'Code_A129','Libelle_A129', 'Code_NC8', 'Libelle_NC8','Code_Pays','Libelle_Pays', 'Valeur', 'Masse', 'USUP']
df_export_new = df_export_new.reindex(columns = new_columns_order)
print(df_export.shape[0])

In [ ]:
df_export.columns

In [ ]:
df_export.loc[df_export['Code_Pays'].isnull()]

### Explication des champs
* Flux : le flux correspond au mouvement de la marchandise. Ce champ est valorisé à I pour
Importation et à E pour Exportation
* Mois : c'est le mois durant lequel a eu lieu le mouvement de marchandise.
* Année: c'est l'année durant laquelle a eu lieu le mouvement de marchandise.
* Code CPF6 : C'est le niveau fin de la nomenclature Classification de produits française (CPF).
La CPF est la nomenclature statistique, nationale et centrale de produits. C'est une nomenclature de
produit et de service. Elle permet aux données du commerce extérieur d'être intégrées avec d'autres
statistiques économiques telles que la production industrielle ou la consommation dans des analyses
économiques tant globales que régionales.
* Code A129 : La nomenclature A129 correspond à une nomenclature de regroupement des niveaux
de la CPF pour répondre aux besoins de la production de données de synthèse pour l'analyse
économique et la diffusion.
* Code NC8 : C'est le code de la nomenclature combinée à 8 chiffres (NC8), utilisée pour les
obligations déclaratives des opérateurs auprès de la douane, permet une connaissance détaillée du
commerce extérieur de la France : elle compte en effet un peu moins de 10 000 rubriques.
* Code Pays : C'est un code à deux caractères alphabétiques attribué à chaque pays par la
Commission des communautés européennes.
     * À l'importation, les marchandises sont relevées au compte du pays d'origine.
     * À l'exportation, les envois sont imputés au compte de la destination finale déclarée.
* Valeur : La valeur des échanges est exprimée en euros.
* Masse : Les masses sont exprimées en kilogrammes.
* USUP : Unité supplémentaire.

In [ ]:
from datetime import datetime
from elasticsearch_dsl import Document, Date, Integer, Keyword, Text
from elasticsearch_dsl.connections import connections

# Define a default Elasticsearch client
connections.create_connection(hosts=['localhost'])



class ExportDouane(Document):
  
    flux = Keyword()
    mois = Integer() 
    annee = Integer()
    code_CPF6 = Keyword()
    code_A129 = Keyword()
    code_NC8 = Keyword()
    code_pays = Keyword()
    valeur = Integer()
    masse = Integer()
    usup = Integer()
    
    class Index:
        name = 'export_douane'

    def save(self, ** kwargs):
        
        return super(ExportDouane, self).save(** kwargs)

    def is_published(self):
        return datetime.now() >= self.published_from

# create the mappings in elasticsearch




In [ ]:
def import_df(df):
    lis = []
    for i in range (df.shape[0]):
        info_export = df.loc[i].tolist() #['Flux', 'Mois', 'Annee', 'Code_CPF6','Code_A129','Code_NC8','Code_Pays','Valeur', 'Masse', 'USUP'] 
         # create and save and article
        export = ExportDouane(meta={'id': i}, tags=['test'])
        export.flux = info_export[0]
        export.mois =  info_export[1]
        export.annee = info_export[2]
        export.code_CPF6 = info_export[3]
        export.code_A129 = info_export[4]
        export.code_NC8 = info_export[5]
        export.code_pays = info_export[6]
        export.valeur = info_export[7]
        export.masse = info_export[8]
        export.usup = info_export[9]
        export.published_from = datetime.now()

        print('Export n°{} importé'.format(i))

    

In [ ]:
df.shape


In [ ]:
lis = df.loc[0].tolist()

In [ ]:
lis


In [ ]:
df1 = df[:20]
df2 = df1.T

In [ ]:
dico=df2.to_dict()
dico

In [ ]:
for i in dico:
    a = json.dumps(dico[i])
    print(a)



In [ ]:
df2.to_json (r'National_2019_Export_test.json') 


In [ ]:
es = elasticsearch.Elasticsearch(['http://localhost:9200/'])

In [ ]:
import numpy as np
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

INDEX="dataframe"
TYPE= "record"

def rec_to_actions(df):
    import json
    for record in df.to_dict(orient="records"):
        yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
        yield (json.dumps(record, default=int))

In [ ]:

e = Elasticsearch()
r = e.bulk(rec_to_actions(df))